# Quantum Conversations: Visualizing Token Generation Paths\n\nThis notebook demonstrates the particle filter approach to tracking and visualizing the \"paths not taken\" in language model generation.

In [ ]:
# Import required libraries
import sys
sys.path.append('..')  # Add parent directory to path to import quantum_conversations

import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

# Force matplotlib to use inline backend
import matplotlib
matplotlib.use('Agg')
%matplotlib inline

from quantum_conversations import ParticleFilter, TokenSequenceVisualizer

# Set up matplotlib style
plt.style.use('default')  # Use default style as seaborn-v0_8 might not be available

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

## Initialize the Particle Filter\n\nWe'll use TinyLlama-1.1B for efficient generation while still getting meaningful results.

In [ ]:
# Initialize particle filter
print("Initializing particle filter...")
print("Note: This will download the TinyLlama model (~2.2GB) on first run.")

# Use 1000 particles as requested
pf = ParticleFilter(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    n_particles=1000,
    temperature=1.0,
    top_k=100,
    top_p=0.95,
    device=device
)

print("Model loaded successfully!")

# Initialize visualizer with settings for many thin particles
viz = TokenSequenceVisualizer(
    tokenizer=pf.tokenizer,
    figsize=(20, 12),
    alpha=0.01,      # Very low alpha for 1000 overlapping paths
    line_width=0.1   # Very thin lines
)

## Define Starter Sequences\n\nWe'll use a variety of prompts with different levels of ambiguity to see how the model explores different paths.

In [ ]:
# Define starter sequences with varying ambiguity\nstarter_sequences = [\n    # High ambiguity - many possible continuations\n    \"It all started when \",\n    \"I couldn't believe that \",\n    \"The most surprising thing was \",\n    \"Nobody expected \",\n    \"In the middle of the night, \",\n    \n    # Medium ambiguity - some constraints\n    \"The scientific experiment revealed \",\n    \"The recipe called for \",\n    \"The weather forecast predicted \",\n    \"The detective discovered \",\n    \"The teacher explained that \",\n    \n    # Low ambiguity - more predictable\n    \"Two plus two equals \",\n    \"The capital of France is \",\n    \"Water freezes at \",\n    \"The sun rises in the \",\n    \"Photosynthesis converts \",\n    \n    # Story beginnings\n    \"Once upon a time, \",\n    \"In a galaxy far, far away, \",\n    \"It was a dark and stormy night when \",\n    \"The door creaked open, revealing \",\n    \"She had never seen anything like \",\n    \n    # Questions and philosophical\n    \"The meaning of life is \",\n    \"What would happen if \",\n    \"The best way to \",\n    \"The secret to happiness is \",\n    \"Time travel would \",\n    \n    # Technical/Code\n    \"def fibonacci(n):\\n    \",\n    \"The algorithm works by \",\n    \"To solve this problem, we \",\n    \"The function returns \",\n    \"class NeuralNetwork:\\n    def __init__(self):\\n        \"\n]

## Generate and Visualize Single Example\n\nLet's start with a detailed look at one example to understand the visualization.

In [ ]:
# Generate particles for one example
example_prompt = "The most surprising thing was "
print(f"Generating particles for: '{example_prompt}'\n")

# Generate with 20 steps as requested
particles = pf.generate(example_prompt, max_new_tokens=20)

# Show the generated sequences
print("\nGenerated sequences:\n")
sequences = pf.get_token_sequences()
# Show only first 5 sequences
for i, (text, log_prob) in enumerate(sequences[:5]):
    print(f"Particle {i+1}: {text}")
    print(f"  Log probability: {log_prob:.3f}\n")
    
print(f"... and {len(sequences)-5} more sequences")

# Find most probable sequence
log_probs = [lp for _, lp in sequences]
most_probable_idx = max(range(len(log_probs)), key=lambda i: log_probs[i])
most_probable_text = sequences[most_probable_idx][0]
print(f"\nMost probable sequence: {most_probable_text}")

In [ ]:
# Create bump plot visualization
fig = viz.visualize_bumplot(
    particles=particles,
    output_path=None,
    max_vocab_display=15,
    color_by='transition_prob',
    show_tokens=True,
    curve_force=0.5,
    prompt=example_prompt
)
plt.show()

## Generate Visualizations for All Starter Sequences\n\nNow let's generate visualizations for all our starter sequences, organizing them by ambiguity level.

In [ ]:
# Create output directory
output_dir = "../../data/derivatives/particle_visualizations"
os.makedirs(output_dir, exist_ok=True)
print(f"Output directory created at: {output_dir}")

# Categories for organization
categories = {
    "high_ambiguity": starter_sequences[0:5],
    "medium_ambiguity": starter_sequences[5:10],
    "low_ambiguity": starter_sequences[10:15],
    "story_beginnings": starter_sequences[15:20],
    "philosophical": starter_sequences[20:25],
    "technical": starter_sequences[25:30]
}

In [ ]:
# Generate visualizations for each category
# We'll process only the first prompt from each category for demonstration
results = {}

for category, prompts in categories.items():
    print(f"\n=== Processing {category} ===")
    category_dir = os.path.join(output_dir, category)
    os.makedirs(category_dir, exist_ok=True)
    
    results[category] = []
    
    # Process only first prompt from each category
    for i, prompt in enumerate(prompts[:1]):
        print(f"Generating 1000 particles for: {prompt[:30]}...")
        
        # Generate particles with 20 tokens
        particles = pf.generate(prompt, max_new_tokens=20)
        
        # Save results
        result = {
            'prompt': prompt,
            'sequences': pf.get_token_sequences(),
            'particles': particles
        }
        results[category].append(result)
        
        # Create safe filename
        safe_prompt = prompt.replace(' ', '_').replace('/', '_').replace('\\n', '_')[:30]
        
        # Generate Sankey diagram
        fig = viz.visualize(
            particles=particles,
            prompt=prompt,
            output_path=os.path.join(category_dir, f"{i:02d}_sankey_{safe_prompt}.png")
        )
        plt.close(fig)
        print(f"  ✓ Generated Sankey diagram")
        
        # Generate heatmap
        fig = viz.visualize_probability_heatmap(
            particles=particles,
            prompt=prompt,
            vocab_size=32000,
            output_path=os.path.join(category_dir, f"{i:02d}_heatmap_{safe_prompt}.png")
        )
        plt.close(fig)
        print(f"  ✓ Generated heatmap")

# Generate visualizations for each category
# We'll process only the first prompt from each category for demonstration
results = {}

for category, prompts in categories.items():
    print(f"\n=== Processing {category} ===")
    category_dir = os.path.join(output_dir, category)
    os.makedirs(category_dir, exist_ok=True)
    
    results[category] = []
    
    # Process only first prompt from each category
    for i, prompt in enumerate(prompts[:1]):
        print(f"Generating 1000 particles for: {prompt[:30]}...")
        
        # Generate particles with 20 tokens
        particles = pf.generate(prompt, max_new_tokens=20)
        
        # Save results
        result = {
            'prompt': prompt,
            'sequences': pf.get_token_sequences(),
            'particles': particles
        }
        results[category].append(result)
        
        # Create safe filename
        safe_prompt = prompt.replace(' ', '_').replace('/', '_').replace('\\n', '_')[:30]
        
        # Generate bump plot visualization
        fig = viz.visualize_bumplot(
            particles=particles,
            output_path=os.path.join(category_dir, f"{i:02d}_bumplot_{safe_prompt}.png"),
            max_vocab_display=15,
            color_by='transition_prob',
            show_tokens=True,
            prompt=prompt
        )
        plt.close(fig)
        print(f"  ✓ Generated bump plot visualization")

In [ ]:
# Calculate divergence metrics\ndef calculate_divergence_metrics(particles):\n    \"\"\"Calculate metrics for path divergence.\"\"\"\n    sequences = [p.tokens for p in particles]\n    n_particles = len(sequences)\n    \n    if n_particles < 2:\n        return {'avg_divergence_point': 0, 'final_diversity': 0}\n    \n    # Find average divergence point\n    min_len = min(len(seq) for seq in sequences)\n    divergence_points = []\n    \n    for i in range(n_particles):\n        for j in range(i + 1, n_particles):\n            # Find where sequences diverge\n            for k in range(min_len):\n                if sequences[i][k] != sequences[j][k]:\n                    divergence_points.append(k)\n                    break\n    \n    avg_divergence = np.mean(divergence_points) if divergence_points else min_len\n    \n    # Calculate final diversity (unique endings)\n    final_tokens = [tuple(seq[-5:]) for seq in sequences if len(seq) >= 5]\n    final_diversity = len(set(final_tokens)) / n_particles if final_tokens else 0\n    \n    return {\n        'avg_divergence_point': avg_divergence,\n        'final_diversity': final_diversity\n    }

In [ ]:
# Analyze divergence by category\ndivergence_analysis = {}\n\nfor category, category_results in results.items():\n    metrics = []\n    for result in category_results:\n        metric = calculate_divergence_metrics(result['particles'])\n        metrics.append(metric)\n    \n    divergence_analysis[category] = {\n        'avg_divergence_point': np.mean([m['avg_divergence_point'] for m in metrics]),\n        'avg_final_diversity': np.mean([m['final_diversity'] for m in metrics])\n    }\n\n# Create visualization\nfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))\n\ncategories_ordered = list(divergence_analysis.keys())\ndivergence_points = [divergence_analysis[c]['avg_divergence_point'] for c in categories_ordered]\ndiversities = [divergence_analysis[c]['avg_final_diversity'] for c in categories_ordered]\n\n# Plot average divergence point\nax1.bar(range(len(categories_ordered)), divergence_points, color='skyblue')\nax1.set_xticks(range(len(categories_ordered)))\nax1.set_xticklabels(categories_ordered, rotation=45, ha='right')\nax1.set_ylabel('Average Divergence Point (tokens)')\nax1.set_title('When Do Paths Diverge?')\nax1.grid(True, alpha=0.3)\n\n# Plot final diversity\nax2.bar(range(len(categories_ordered)), diversities, color='lightcoral')\nax2.set_xticks(range(len(categories_ordered)))\nax2.set_xticklabels(categories_ordered, rotation=45, ha='right')\nax2.set_ylabel('Final Diversity Score')\nax2.set_title('How Different Are Final Outputs?')\nax2.grid(True, alpha=0.3)\n\nplt.tight_layout()\nplt.savefig(os.path.join(output_dir, 'divergence_analysis.png'), dpi=300, bbox_inches='tight')\nplt.show()

## Interactive Exploration\n\nTry your own prompts to see how the model explores different paths!

In [ ]:
# Interactive prompt exploration\ndef explore_prompt(prompt, n_particles=10, max_tokens=30, temperature=0.8):\n    \"\"\"Explore a custom prompt with particle filtering.\"\"\"\n    # Update particle filter settings\n    pf.n_particles = n_particles\n    pf.temperature = temperature\n    \n    # Generate\n    print(f\"Generating {n_particles} particles for: '{prompt}'\")\n    particles = pf.generate(prompt, max_new_tokens=max_tokens)\n    \n    # Show sequences\n    print(\"\\nGenerated sequences:\")\n    for i, (text, log_prob) in enumerate(pf.get_token_sequences()):\n        print(f\"\\nParticle {i+1}:\")\n        print(f\"  Text: {text}\")\n        print(f\"  Log prob: {log_prob:.3f}\")\n    \n    # Visualize\n    fig = viz.visualize(particles=particles, prompt=prompt)\n    plt.show()\n    \n    return particles\n\n# Example usage\ncustom_particles = explore_prompt(\n    \"The future of AI will \",\n    n_particles=8,\n    max_tokens=25,\n    temperature=1.0\n)

# Interactive prompt exploration
def explore_prompt(prompt, n_particles=10, max_tokens=30, temperature=0.8):
    """Explore a custom prompt with particle filtering."""
    # Update particle filter settings
    pf.n_particles = n_particles
    pf.temperature = temperature
    
    # Generate
    print(f"Generating {n_particles} particles for: '{prompt}'")
    particles = pf.generate(prompt, max_new_tokens=max_tokens)
    
    # Show sequences
    print("\nGenerated sequences:")
    for i, (text, log_prob) in enumerate(pf.get_token_sequences()):
        print(f"\nParticle {i+1}:")
        print(f"  Text: {text}")
        print(f"  Log prob: {log_prob:.3f}")
    
    # Visualize with bump plot
    fig = viz.visualize_bumplot(
        particles=particles,
        max_vocab_display=15,
        color_by='transition_prob',
        show_tokens=True,
        prompt=prompt
    )
    plt.show()
    
    return particles

# Example usage
custom_particles = explore_prompt(
    "The future of AI will ",
    n_particles=8,
    max_tokens=25,
    temperature=1.0
)